# Topic Modeling based on Digitised Volumes of theatrical English, Scottish, and Irish playbills between 1600 - 1902 from data.bl.uk

Topic Models are a type of statistical language models used for discovering hidden structure in a collection of texts. 

This example is based on the a dataset that comprises 264 volumes of digitised theatrical playbills published between 1660 – 1902 (mostly 19th century) from England, Scotland, Wales and Ireland. Digitised from the British Library's physical collection of over 500 volumes of playbills. The dataset contains text files (.TXT) in Optical Character Recognition (OCR) format. More information about the dataset at https://data.bl.uk/playbills/

## Setting up things

In [1]:
import sys
import requests
import pandas as pd
import re
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/gustavo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/gustavo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Reading the CSV file

**Note:** the original dataset did not include a CSV file. It was generated from the Excel file.

In [2]:
# Read data into playbills
playbills = pd.read_csv('playbills-ocr-text/playbills.csv', encoding='iso-8859-1')

# Print head
playbills.head()

,Ingestion Order,Shelf Mark,Volume,PID,Create Date,File Name (.PDF),Path,File Size (MB),LSID
0,2,Playbills 1,Playbills 1,373904,3 de sep de 10,015376973_313_Playbills 1,/2010/09/03/file_1/373904,"156,38",3f9b9a08
1,12,Playbills 10,Playbills 10,379308,7 de sep de 10,014933523_1305_Playbills 10,/2010/09/07/file_4/379308,"185,7",3f9c9e0f
2,101,Playbills 100,Playbills 100,435983,21 de sep de 10,015176783_1333_Playbills 100,/2010/09/21/file_6/435983,"294,12",400ce74c
3,103,Playbills 101,Playbills 101 (1),436802,21 de sep de 10,015176791_2150_Playbills 101 (1),/2010/09/21/file_7/436802,"70,98",400ce5c3
4,105,Playbills 101,Playbills 101 (2),437629,21 de sep de 10,015176791_2975_Playbills 101 (2),/2010/09/21/file_8/437629,"75,05",400c8a20


## Data cleaning

Since the goal of this analysis is to perform topic modeling, we will focus on the text data from each register, and remove other metadata columns that are not necessary.

In [3]:
# Remove the columns
playbills = playbills.drop(columns=['Ingestion Order', 'Shelf Mark', 'PID', 'Path', 'File Name (.PDF)', 'File Size (MB)'], axis=1)# Print out the first rows of papers
playbills.head()

,Volume,Create Date,LSID
0,Playbills 1,3 de sep de 10,3f9b9a08
1,Playbills 10,7 de sep de 10,3f9c9e0f
2,Playbills 100,21 de sep de 10,400ce74c
3,Playbills 101 (1),21 de sep de 10,400ce5c3
4,Playbills 101 (2),21 de sep de 10,400c8a20


## Reading the files and extracting the text

In [4]:
for index,row in playbills.iterrows():
    
    try:
        file = "playbills-ocr-text/lsidyv"+ row['LSID'] +".txt";
        f = open(file, "r")
        text = f.read()
        
        playbills.loc[index, 'original_text'] = text
                
    except:
        print("An exception occurred", sys.exc_info()[0]) 
        playbills.loc[index, 'original_text'] = ''

An exception occurred <class 'FileNotFoundError'>
An exception occurred <class 'FileNotFoundError'>
An exception occurred <class 'FileNotFoundError'>
An exception occurred <class 'FileNotFoundError'>
An exception occurred <class 'FileNotFoundError'>


## Reviewing the content of the files

In [5]:
playbills.head()

,Volume,Create Date,LSID,original_text
0,Playbills 1,3 de sep de 10,3f9b9a08,"﻿\n\n\n\n\n\n\n■1\n\nDRURY LAN E,\nBy hi» M A ..."
1,Playbills 10,7 de sep de 10,3f9c9e0f,"﻿- )♦ I >\nPZ,AYß .\n¿Ij¿/P¿/ ¿fi. !\nThe fh y..."
2,Playbills 100,21 de sep de 10,400ce74c,")\n\n T H E A T R E ROYAL,\n\nCOYEST-GARDEN.\n..."
3,Playbills 101 (1),21 de sep de 10,400ce5c3,"TilEATRE ROYAL,\n\nCovent- €rarde>\nThe Public..."
4,Playbills 101 (2),21 de sep de 10,400c8a20,"Theatre Royal, CoYent-Gardeii\nThis ^ireseut S..."


## Remove punctuation/lower casing/stopwords

Next, let’s perform a simple preprocessing on the content to make them more amenable for analysis, and reliable results. To do that, we’ll use a regular expression to remove any punctuation, lowercase the text, remove stopwords and then remove non English words since the OCR may have some errors.

We use wordnet to verify if the word exists. We also have added some specific stopwords to enhance the performance.

The initial_clean function performs an initial clean by removing punctuations, uppercase text etc. 

In [6]:
def initial_clean(text):
     """
     Function to clean text-remove punctuations, lowercase text etc.    
     """
     # remove_digits and special chars   
     text = re.sub("[^a-zA-Z ]", "", text)
    
     text = text.lower() # lower case text
     text = nltk.word_tokenize(text)
     return text

The next function stem_words() stems the words to its base forms to reduce variant forms of words.

In [15]:
stemmer = PorterStemmer()
def stem_words(text):
     """
     Function to stem words
     """
     #try:
     text = [stemmer.stem(word) for word in text]
     text = [word for word in text if len(word) > 2] # no single letter words
     #except IndexError:
     #    pass
     return text  

Let's see an example

In [16]:
some_words = "William Shakespeare was perhaps the most famous author"
some_words_tokens = nltk.word_tokenize(some_words)
print(stem_words(some_words_tokens))

['william', 'shakespear', 'perhap', 'the', 'most', 'famou', 'author']


We will use wordnet to remove non existent words. Due to the text provided in the dataset many words are not existent. We will encrease the performance by removing non existent words.

In [17]:
def remove_non_english_words(text):
    filtered_text = [] 
    
    for token in text:

        if len(token) == 1:
            continue
        elif token in stop_words:
            continue
        elif not wordnet.synsets(token):
            #Not an English Word
            continue
        else:
            #English Word
            filtered_text.append(token)
    return filtered_text

In general, common  words known as *stopwords* are removed from text since they could be considered as noise when used in text algorithms.

In [18]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['news', 'say','use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do','took','time','year',
'done', 'try', 'many', 'some','nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line','even', 'also', 'may', 'take', 'come', 'new','said', 'like','people'])
def remove_stop_words(text):
     return [word for word in text if word not in stop_words]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gustavo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We create a function to perform the whole process

In [19]:
def apply_all(text):
     """
     This function applies all the functions above into one
     """
     return stem_words(remove_stop_words(remove_non_english_words(initial_clean(text))))

Finallly, we process the original text by using the function apply.

In [20]:
# clean reviews and create new column "tokenized" 
import time   
t1 = time.time()   
playbills['tokenized_text'] = playbills['original_text'].apply(apply_all)    
t2 = time.time()  
print("Time to clean and tokenize", len(playbills), "reviews:", (t2-t1)/60, "min") #Time to clean and tokenize

Time to clean and tokenize 264 reviews: 9.853082410494487 min


## Checking the result

In [21]:
playbills.head()

,Volume,Create Date,LSID,original_text,tokenized_text
0,Playbills 1,3 de sep de 10,3f9b9a08,"﻿\n\n\n\n\n\n\n■1\n\nDRURY LAN E,\nBy hi» M A ...","[lan, testi, compli, theatr, royal, day, play,..."
1,Playbills 10,7 de sep de 10,3f9c9e0f,"﻿- )♦ I >\nPZ,AYß .\n¿Ij¿/P¿/ ¿fi. !\nThe fh y...","[rip, air, aft, fist, royal, public, respect, ..."
2,Playbills 100,21 de sep de 10,400ce74c,")\n\n T H E A T R E ROYAL,\n\nCOYEST-GARDEN.\n...","[present, januari, opera, conrad, caspar, blac..."
3,Playbills 101 (1),21 de sep de 10,400ce5c3,"TilEATRE ROYAL,\n\nCovent- €rarde>\nThe Public...","[public, respect, inform, monday, princip, per..."
4,Playbills 101 (2),21 de sep de 10,400c8a20,"Theatre Royal, CoYent-Gardeii\nThis ^ireseut S...","[theatr, royal, saturday, jan, act, sheridan, ..."


## Create Gensim Dictionary and Corpus

Topic modeling using LDA are based on the dictionary and the corpus. This example is based on gensim library for building both.

In [22]:
# LDA
import gensim
from gensim import corpora, models, similarities 

In [23]:
tokenized = playbills['tokenized_text']

#Creating term dictionary of corpus, where each unique term is assigned an index.
dictionary = corpora.Dictionary(tokenized)
#Filter terms which occurs in less than 1 review and more than 80% of the reviews.
dictionary.filter_extremes(no_below=1, no_above=0.8)
#convert the dictionary to a bag of words corpus 
corpus = [dictionary.doc2bow(tokens) for tokens in tokenized]
#print(corpus[:1])

[[(0, 1), (1, 2), (2, 2), (3, 5), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 4), (13, 3), (14, 20), (15, 1), (16, 3), (17, 3), (18, 2), (19, 7), (20, 3), (21, 1), (22, 16), (23, 1), (24, 9), (25, 1), (26, 1), (27, 1), (28, 14), (29, 1), (30, 1), (31, 1), (32, 4), (33, 6), (34, 10), (35, 2), (36, 1), (37, 2), (38, 3), (39, 1), (40, 1), (41, 2), (42, 120), (43, 7), (44, 1), (45, 2), (46, 2), (47, 1), (48, 4), (49, 2), (50, 2), (51, 2), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 8), (58, 83), (59, 8), (60, 1), (61, 22), (62, 2), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 2), (69, 3), (70, 2), (71, 1), (72, 1), (73, 5), (74, 2), (75, 1), (76, 1), (77, 2), (78, 1), (79, 11), (80, 1), (81, 6), (82, 2), (83, 1), (84, 1), (85, 3), (86, 2), (87, 3), (88, 8), (89, 8), (90, 1), (91, 6), (92, 2), (93, 5), (94, 4), (95, 2), (96, 1), (97, 2), (98, 2), (99, 1), (100, 2), (101, 1), (102, 3), (103, 1), (104, 1), (105, 1), (106, 3), (107, 2), (108, 3), (109, 10)

## Building the Topic Model

In this step, num_topics is the number of topics to be created and passes correspond to the number of times to iterate through the entire corpus. By running the LDA algorithm we get the topics as a result.

In [25]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

#LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 5, id2word=dictionary, passes=15)
ldamodel.save('model_combined.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
   print(topic)

(0, '0.003*"interest" + 0.003*"laughabl" + 0.003*"fairi" + 0.003*"precis"')
(1, '0.009*"bologna" + 0.008*"money" + 0.007*"mailer" + 0.007*"rex"')
(2, '0.009*"webster" + 0.007*"success" + 0.006*"guinea" + 0.005*"strickland"')
(3, '0.007*"cooper" + 0.007*"kean" + 0.006*"applaus" + 0.006*"webster"')
(4, '0.005*"miller" + 0.005*"circl" + 0.004*"griev" + 0.004*"fairi"')


This output shows the 5 topics created and the 4 words within each topic which best describes them. From the above output we could guess that each topic and their corresponding words revolve around a common theme (For e.g., Topic 2 is related to bologna and money).